In [39]:
!pip3 install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 2.3 MB/s eta 0:00:0000:0100:01


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
raw_data = pd.read_csv('winequality-white.csv', sep='\t', header=None)  # Using tab as separator

# Split the first column (which contains all attributes) by semicolons
df_split = raw_data[0].str.split(';', expand=True)

# Extract the header from the first row and create a list of attribute names
attribute_names = df_split.iloc[0].str.replace('"', '', regex=False).str.strip()

# Create a DataFrame from the remaining rows (excluding the first one)
df = df_split[1:]

# Assign the header to the DataFrame
df.columns = attribute_names

# Convert only the numeric rows to float, using `pd.to_numeric` to handle conversion
df = df.apply(pd.to_numeric, errors='coerce')
# Check the resulting DataFrame
print("Split DataFrame shape:", df.shape)

0     fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
1               7.0              0.27         0.36            20.7      0.045   
2               6.3              0.30         0.34             1.6      0.049   
3               8.1              0.28         0.40             6.9      0.050   
4               7.2              0.23         0.32             8.5      0.058   
5               7.2              0.23         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
4894            6.2              0.21         0.29             1.6      0.039   
4895            6.6              0.32         0.36             8.0      0.047   
4896            6.5              0.24         0.19             1.2      0.041   
4897            5.5              0.29         0.30             1.1      0.022   
4898            6.0              0.21         0.38             0.8      0.020   

0     free sulfur dioxide  

In [66]:
# Assuming the last column is the target variable
# X is the features, Y is the target
X = df.iloc[:,:-1]   # All columns except the last one
Y = df.iloc[:, -1]   # The last column is the target
#print(Y)
# Check the split data
print("Feature Matrix (X) shape:", X.shape)
print("Target Vector (Y) shape:", Y.shape)

# Define your percentages
train = 0.7  # 70% for training
# validation = 0.15 # 15% for validation
# The remaining percentage for testing will be (100 - t - v)% = 15%

# Proceed with splitting the dataset
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size = (1-train), random_state = 1)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state= 1)

Feature Matrix (X) shape: (4898, 11)
Target Vector (Y) shape: (4898,)


In [69]:
# Finding the best alpha for L2 and L1 regularization 
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error

# Define a list of candidate regularization parameters (alpha values)
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

# Fine best alpha for L2 and L1 regularization
def find_best_alpha(X_train, Y_train, X_valid, Y_valid, alphas, reg_type):
    """
    Find the best alpha for L2 regularization using Ridge regression.
    
    Parameters:
    X_train (array-like): Training feature set.
    Y_train (array-like): Training target set.
    X_valid (array-like): Validation feature set.
    Y_valid (array-like): Validation target set.
    alphas (list): List of candidate alpha values for regularization.
    reg_type (str): Type of regularization ('l1' for Lasso, 'l2' for Ridge).
    
    Returns:
    float: The best alpha value with the lowest validation error.
    """
    validation_errors = []
    for alpha in alphas:
        if reg_type == 'l2':
            # Create the Ridge model with current regularization parameter
            model = Ridge(alpha=alpha)
        elif reg_type == 'l1':
            # Create the Lasso model with current regularization parameter
            model = Lasso(alpha=alpha)
        else:
            raise ValueError("reg_type must be 'l1' or 'l2'.")
            
        # Train the model on the training set
        model.fit(X_train, Y_train)
        
        # Predict on the validation set
        Y_valid_hat = model.predict(X_valid)
        
        # Compute the validation error (MSE in this case)
        mse = mean_squared_error(Y_valid, Y_valid_hat)
        validation_errors.append(mse)

    # Find the best alpha (with the lowest validation error)
    best_alpha = alphas[np.argmin(validation_errors)]
    return best_alpha

In [70]:
# Batch Gradient Descent without Regularization
def batch_gradient_descent(X_train, Y_train, X_valid, Y_valid, X_test, Y_test, learning_rate, epochs, reg_type='none'):
    m, n = X_train.shape
    W = np.zeros(n + 1) # Weight vector initializatio- we started with a zero vector
    X_train = np.c_[np.ones((m, 1)), X_train] # Add bias term
    X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]  # Add bias term for test set
    X_valid = np.c_[np.ones((X_valid.shape[0], 1)), X_valid]  # Add bias term for test set

    best_alpha = None
    if reg_type in ['l1', 'l2']:
        best_alpha = find_best_alpha(X_train, Y_train, X_valid, Y_valid, alphas, reg_type)
        print(f"Best regularization parameter (alpha) for {reg_type}: {best_alpha}")

    for epoch in range (epochs):
        Y_hat = np.dot(X_train,W) # Compute predictions
        mse = np.mean((Y_train - Y_hat) ** 2) # Mean Squared Error

        if reg_type == 'l2':
            cost = mse + best_alpha * np.sum(W ** 2)  # Mean Squared Error
            gradient = (2/m) * np.dot(X_train.T, (Y_hat - Y_train)) + 2*best_alpha*W # Gradient
        elif reg_type == 'l1':
            cost = mse + best_alpha * np.sum(W ** 2)  # Mean Squared Error
            gradient = (2/m) * np.dot(X_train.T, (Y_hat - Y_train)) + best_alpha*np.sign(W) # Gradient
        else:
            cost = mse
            gradient = (2/m) * np.dot(X_train.T, (Y_hat - Y_train)) # Gradient
        
        W = W-learning_rate * gradient  # Update weights

        if epoch % 10 == 0:
            print(f'Epoch {epoch}: MSE = {mse}')
            
    # Evaluate on the test set
    if reg_type == 'l2':
        test_model = Ridge(alpha=best_alpha, fit_intercept=False)
    elif reg_type == 'l1':
        test_model = Lasso(alpha=best_alpha, fit_intercept=False)
    else:
        test_model = None  # No model for testing if there's no regularization

    if test_model:
        test_model.fit(X_train[:, 1:], Y_train)
        Y_test_hat = test_model.predict(X_test[:, 1:])
        test_error = mean_squared_error(Y_test, Y_test_hat)
        print(f"Test error with best alpha: {test_error}")
        
    return W

In [71]:
# Batch without Regularization
W_batch = batch_gradient_descent(X_train, Y_train, X_valid, Y_valid, X_test, Y_test, learning_rate=0.00001, epochs=8000, reg_type='none')
print("Weights from Batch GD without Regularization:", W_batch)
#We tried 100,000 interations and found that after almost 8000 iterations, MSE was almost stagnant around 0.67. 

Epoch 0: MSE = 35.236289381563594
Epoch 10: MSE = 4.1802724463143885
Epoch 20: MSE = 4.1537305161725575
Epoch 30: MSE = 4.1275762153136215
Epoch 40: MSE = 4.101619657734249
Epoch 50: MSE = 4.075859330524654
Epoch 60: MSE = 4.050293733852131
Epoch 70: MSE = 4.024921379704523
Epoch 80: MSE = 3.9997407917793746
Epoch 90: MSE = 3.9747505053752192
Epoch 100: MSE = 3.949949067284896
Epoch 110: MSE = 3.925335035690818
Epoch 120: MSE = 3.9009069800620764
Epoch 130: MSE = 3.8766634810533214
Epoch 140: MSE = 3.8526031304053276
Epoch 150: MSE = 3.828724530847181
Epoch 160: MSE = 3.80502629600001
Epoch 170: MSE = 3.7815070502822015
Epoch 180: MSE = 3.758165428816046
Epoch 190: MSE = 3.7350000773357426
Epoch 200: MSE = 3.7120096520967274
Epoch 210: MSE = 3.6891928197862573
Epoch 220: MSE = 3.666548257435222
Epoch 230: MSE = 3.644074652331123
Epoch 240: MSE = 3.6217707019321876
Epoch 250: MSE = 3.599635113782578
Epoch 260: MSE = 3.577666605428658
Epoch 270: MSE = 3.5558639043362805
Epoch 280: MSE = 

In [72]:
# Batch L2 Regularization
#W_batch_l2 = batch_gradient_descent_l2(X_train, Y_train, learning_rate=0.00001, epochs=10000)
W_batch = batch_gradient_descent(X_train, Y_train, X_valid, Y_valid, X_test, Y_test, learning_rate=0.00001, epochs=10000, reg_type='l2')
print("Weights from Batch GD with L2 Regularization:", W_batch_l2)
# We tried 100,000 interations and found that after almost 10000 iterations, value of the cost function was almost stagnant around 0.68#Define learning rate

Best regularization parameter (alpha) for l2: 0.0001
Epoch 0: MSE = 35.236289381563594
Epoch 10: MSE = 4.180272447214731
Epoch 20: MSE = 4.153730517174711
Epoch 30: MSE = 4.127576217589772
Epoch 40: MSE = 4.101619661789895
Epoch 50: MSE = 4.075859336853967
Epoch 60: MSE = 4.0502937429397345
Epoch 70: MSE = 4.02492139202562
Epoch 80: MSE = 3.9997408077998733
Epoch 90: MSE = 3.9747505255518507
Epoch 100: MSE = 3.9499490920653413
Epoch 110: MSE = 3.925335065513823
Epoch 120: MSE = 3.900907015357569
Epoch 130: MSE = 3.876663522242531
Epoch 140: MSE = 3.8526031779008996
Epoch 150: MSE = 3.828724585053289
Epoch 160: MSE = 3.80502635731247
Epoch 170: MSE = 3.7815071190885847
Epoch 180: MSE = 3.758165505495785
Epoch 190: MSE = 3.7350001622602433
Epoch 200: MSE = 3.712009745629476
Epoch 210: MSE = 3.6891929222829276
Epoch 220: MSE = 3.6665483692437775
Epoch 230: MSE = 3.6440747737919232
Epoch 240: MSE = 3.6217708333780925
Epoch 250: MSE = 3.599635255539048
Epoch 260: MSE = 3.5776667578138523
Ep

In [73]:
# Batch L1 Regularization
#W_batch_l1 = batch_gradient_descent_l1(X_train, Y_train, learning_rate=0.00001, epochs=11000)
W_batch = batch_gradient_descent(X_train, Y_train, X_valid, Y_valid, X_test, Y_test, learning_rate=0.00001, epochs=11000, reg_type='l1')
print("Weights from Batch GD with L1 Regularization:", W_batch_l1)
# We tried 100,000 interations and found that after almost 11000 iterations, value of the cost function was almost stagnant around 0.68

Best regularization parameter (alpha) for l1: 0.001
Epoch 0: MSE = 35.236289381563594
Epoch 10: MSE = 4.18027490878539
Epoch 20: MSE = 4.153735417887972
Epoch 30: MSE = 4.127583655370923
Epoch 40: MSE = 4.101629608533638
Epoch 50: MSE = 4.075871764430248
Epoch 60: MSE = 4.050308623500023
Epoch 70: MSE = 4.024938697998377
Epoch 80: MSE = 3.999760511885496
Epoch 90: MSE = 3.974772600717866
Epoch 100: MSE = 3.949973511541812
Epoch 110: MSE = 3.925361802788966
Epoch 120: MSE = 3.900936044173568
Epoch 130: MSE = 3.87669481659153
Epoch 140: MSE = 3.852636712021167
Epoch 150: MSE = 3.8287603334255444
Epoch 160: MSE = 3.8050642946563573
Epoch 170: MSE = 3.781547220359287
Epoch 180: MSE = 3.7582077458807714
Epoch 190: MSE = 3.7350445171761395
Epoch 200: MSE = 3.712056190719047
Epoch 210: MSE = 3.6892414334121746
Epoch 220: MSE = 3.6665989224991327
Epoch 230: MSE = 3.6441273454775427
Epoch 240: MSE = 3.6218254000132335
Epoch 250: MSE = 3.59969179385554
Epoch 260: MSE = 3.5777252447536396
Epoch 2

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.607e+02, tolerance: 1.208e+01
  model = cd_fast.enet_coordinate_descent(
